## 모듈 불러오기

In [1]:
import pandas as pd

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from collections import Counter
from collections import defaultdict

import re

from tqdm import tqdm


## pip install 
- 필요시 마크다운 해제 후 사용

## Option
- 1. 출력 셀 전체 보기

In [1]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

NameError: name 'pd' is not defined

## 파일 불러오기
- 1. 'cs.' 으로 시작하는 카테고리 (arxiv data)

In [8]:
docs_path1= 'arxiv_cs.csv'
docs1 = pd.read_csv(docs_path1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_31592\2386913889.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  docs1 = pd.read_csv(docs_path1)


In [75]:
docs2_path = 'vscode/paperswithcode.csv'
docs2 = pd.read_csv(docs2_path)

In [81]:
docs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   method    1928 non-null   object
 1   category  1928 non-null   object
 2   model     1926 non-null   object
dtypes: object(3)
memory usage: 45.3+ KB


In [87]:
docs2.sample(n=10, random_state=16)

,method,category,model
20,Computer Vision,Image Models,DeiT
1875,Graphs,Graph Models,GAP-Layer
1423,Natural Language Processing,Autoencoding Transformers,DeBERTa
1820,Graphs,Graph Models,Graph Transformer
1704,Audio,Audio Model Blocks,WaveGrad DBlock
920,Computer Vision,3D Reconstruction,ARCH
292,Computer Vision,Generative Models,GAN
1879,Graphs,Graph Embeddings,TuckER
1633,Reinforcement Learning,Distributed Reinforcement Learning,TorchBeast
698,Computer Vision,Vision Transformers,Shuffle-T


## Main

### 목적
- 1) title & abstract의 키워드를 통해 categories 예측
    - 방법1) tokenizing -> 각 카테고리별 빈도수 높은 단어 추출
    - 방법2) paperswithcode의 카테고리별 모델명 불러오기
    

### 1. 데이터 분석

In [5]:
docs1. head(3)

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 13:06:50 GMT","Sun, 1 Apr 2007 13:06:50 GMT",Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...
1,49,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 18:53:13 GMT","Sun, 1 Apr 2007 18:53:13 GMT",Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...
2,61,704.0062,"Rastislav \v{S}r\'amek, Bro\v{n}a Brejov\'a, T...",On-line Viterbi Algorithm and Its Relationship...,cs.DS,"Sat, 31 Mar 2007 23:52:33 GMT","Sat, 31 Mar 2007 23:52:33 GMT",Tom\'a\v{s} Vina\v{r},NaN,Algorithms in Bioinformatics: 7th Internationa...,10.1007/978-3-540-74126-8_23,NaN,NaN,"In this paper, we introduce the on-line Vite..."


In [8]:
docs1.isnull().sum()
# 결측치 다수 : comments, journal-ref, doi, report-no 

Unnamed: 0           0
id                   0
authors              0
title                0
categories           0
created              0
last_updated         0
submitter          128
comments        170374
journal-ref     382078
doi             365064
report-no       431145
license           9203
abstract             0
dtype: int64

In [7]:
docs1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438727 entries, 0 to 438726
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    438727 non-null  int64 
 1   id            438727 non-null  object
 2   authors       438727 non-null  object
 3   title         438727 non-null  object
 4   categories    438727 non-null  object
 5   created       438727 non-null  object
 6   last_updated  438727 non-null  object
 7   submitter     438599 non-null  object
 8   comments      268353 non-null  object
 9   journal-ref   56649 non-null   object
 10  doi           73663 non-null   object
 11  report-no     7582 non-null    object
 12  license       429524 non-null  object
 13  abstract      438727 non-null  object
dtypes: int64(1), object(13)
memory usage: 46.9+ MB


### 2. tokenizing

1) cs.AI

In [10]:
cs_ai_docs = docs1[docs1['categories'].str.contains('cs.AI')]

In [12]:
cs_ai_docs.head(3)

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 13:06:50 GMT","Sun, 1 Apr 2007 13:06:50 GMT",Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...
1,49,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 18:53:13 GMT","Sun, 1 Apr 2007 18:53:13 GMT",Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...
12,303,704.0304,Carlos Gershenson,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,"Tue, 3 Apr 2007 02:08:48 GMT","Wed, 13 Oct 2010 19:49:16 GMT",Carlos Gershenson,"16 pages. Extended version, three more laws of...","Minai, A., Braha, D., and Bar-Yam, Y., eds. Un...",10.1007/978-3-642-18003-3_10,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,This paper discusses the benefits of describ...


In [13]:
cs_ai_abstract = cs_ai_docs.abstract

In [15]:
# tokenize를 통한 abstract의 corpus 추출
tokenized_cs_ai_abstract = [nltk.word_tokenize(i) for i in cs_ai_abstract]
tokenized_cs_ai_abstract

[['The',
  'intelligent',
  'acoustic',
  'emission',
  'locator',
  'is',
  'described',
  'in',
  'Part',
  'I',
  ',',
  'while',
  'Part',
  'II',
  'discusses',
  'blind',
  'source',
  'separation',
  ',',
  'time',
  'delay',
  'estimation',
  'and',
  'location',
  'of',
  'two',
  'simultaneously',
  'active',
  'continuous',
  'acoustic',
  'emission',
  'sources',
  '.',
  'The',
  'location',
  'of',
  'acoustic',
  'emission',
  'on',
  'complicated',
  'aircraft',
  'frame',
  'structures',
  'is',
  'a',
  'difficult',
  'problem',
  'of',
  'non-destructive',
  'testing',
  '.',
  'This',
  'article',
  'describes',
  'an',
  'intelligent',
  'acoustic',
  'emission',
  'source',
  'locator',
  '.',
  'The',
  'intelligent',
  'locator',
  'comprises',
  'a',
  'sensor',
  'antenna',
  'and',
  'a',
  'general',
  'regression',
  'neural',
  'network',
  ',',
  'which',
  'solves',
  'the',
  'location',
  'problem',
  'based',
  'on',
  'learning',
  'from',
  'example

In [16]:
filtered_tokens = []
stop_words = set(stopwords.words('english')) # 불용어 제거

for abstract_tokens in tokenized_cs_ai_abstract:
    # isalnum() : 숫자&문자 
    # 불용어가 아니고 숫자&문자로 이뤄진 단어만 추출
    cleaned_tokens = [token.lower() for token in abstract_tokens if token.isalnum() and token.lower() not in stop_words]
    filtered_tokens.extend(cleaned_tokens)

# 단어별 빈도수 계산
counter = Counter(filtered_tokens)
most_common_words = counter.most_common(30)

In [17]:
most_common_words

[('learning', 62393),
 ('model', 50688),
 ('data', 47126),
 ('models', 41222),
 ('paper', 30739),
 ('performance', 28344),
 ('methods', 26934),
 ('using', 25795),
 ('propose', 25724),
 ('show', 25720),
 ('results', 25458),
 ('approach', 25010),
 ('method', 24419),
 ('problem', 23231),
 ('based', 23017),
 ('training', 22271),
 ('tasks', 22247),
 ('proposed', 21646),
 ('information', 20474),
 ('different', 20430),
 ('network', 20196),
 ('neural', 20028),
 ('new', 19836),
 ('knowledge', 19768),
 ('also', 19719),
 ('however', 18847),
 ('framework', 18501),
 ('algorithm', 18478),
 ('work', 18008),
 ('two', 17972)]

In [19]:
# 결론 
# tokenize로 유의미한 결과를 도출할 수 없다.
# -> 해당 결과로 category를 분류하기엔 무리가 있다고 판단

### 3. 모델에 따른 카테고리화 ex. VGG

1) cs.AI

In [21]:
# abstract에서 VGG(대소문자 구분X)를 포함하는 행 찾기
filtered_cs_ai_docs = cs_ai_docs[cs_ai_docs['abstract'].str.contains(re.compile("VGG", re.I))]
display(len(cs_ai_docs))
display(len(filtered_cs_ai_docs))

64481

198

In [22]:
tmp = filtered_cs_ai_docs.loc[:,"abstract"]
tmp.iloc[0]
# 해당 구문에 VGGNET 이 들어가 있음

'  In this paper, we evaluate convolutional neural network (CNN) features using\nthe AlexNet architecture and very deep convolutional network (VGGNet)\narchitecture. To date, most CNN researchers have employed the last layers\nbefore output, which were extracted from the fully connected feature layers.\nHowever, since it is unlikely that feature representation effectiveness is\ndependent on the problem, this study evaluates additional convolutional layers\nthat are adjacent to fully connected layers, in addition to executing simple\ntuning for feature concatenation (e.g., layer 3 + layer 5 + layer 7) and\ntransformation, using tools such as principal component analysis. In our\nexperiments, we carried out detection and classification tasks using the\nCaltech 101 and Daimler Pedestrian Benchmark Datasets.\n'

2) cs.CV

In [29]:
cs_cv_docs = docs1[docs1['categories'].str.contains('cs.CV')]
filtered_cs_cv_docs = cs_cv_docs[cs_cv_docs['abstract'].str.contains(re.compile("VGG", re.I))]
len(filtered_cs_cv_docs)

1071

3) cs.DM (Discrete Mathmatics)

In [25]:
cs_dm_docs = docs1[docs1['categories'].str.contains('cs.DM')]
filtered_cs_dm_docs = cs_dm_docs[cs_dm_docs['abstract'].str.contains(re.compile("VGG", re.I))]
len(filtered_cs_dm_docs)

1

In [28]:
# 결론 
# tokenize(방법1)에 비해 유의미한 결론을 도출할 수 있었음
# 대분류, 즉 CV와 NLP에서 사용하는 모델은 차이가 있기에 분류 기준으로서 적합하다고 판단
# paperswithcode의 카테고리별 모델을 불러와서 위의 코드를 실행해도 좋을 듯

### 4. 모델에 따른 카테고리화 ex. VGG, ResNet, AlexNet

1) cs.AI

In [30]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_ai_docs[cs_ai_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

Resnet: 558
VGG: 198
AlexNet: 57


2) cs.CV

In [31]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_cv_docs[cs_cv_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

Resnet: 2411
VGG: 1071
AlexNet: 342


3) cs.DM

In [32]:
# 모델 리스트
model_list = ["Resnet", "VGG", "AlexNet"]

model_counts = defaultdict(int)

for model in model_list:
    pattern = re.compile(model, re.I)
    filtered_docs = cs_dm_docs[cs_dm_docs['abstract'].str.contains(pattern)]
    model_counts[model] = len(filtered_docs) # key : value

for model, count in model_counts.items():
    print(f"{model}: {count}")

Resnet: 1
VGG: 1
AlexNet: 0


### 5. paperswithcode dataset (크롤링)

In [36]:

'''
1. see all categories (click) 
    기존 : (6,3) : general 제외, graph 예외처리 
    변경 : (4,3)클릭 위치 변경을 통한 예외를 없애줌 
<기존>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(6) > a    
    
<변경>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(4) > div > h4 > a
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(7) > div > h4 > a


2. method (고정값)
body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1


3. see all models (click)
    (3,3)

body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child(3) > a

4. categories (고정값)
body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1

5. models
    (1,1) : category에 해당하는 모델이 없는 경우(예외 처리)
    #methodsTable > tbody > tr:nth-child(1) > td:nth-child(1) > div.method-image > a
    #methodsTable > tbody > tr:nth-child(2) > td:nth-child(1) > div.method-image > a
'''

'\n1. see all categories (click) \n    기존 : (6,3) : general 제외, graph 예외처리 \n    변경 : (4,3)클릭 위치 변경을 통한 예외를 없애줌 \n<기존>\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(6) > a    \n    \n<변경>\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(4) > div > h4 > a\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(7) > div > h4 > a\n\n\n2. method (고정값)\nbody > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1\n\n\n3. see all models (click)\n    (3,3)\n\nbody > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child(3) > a\n\n4. categories (고정값)\nbody > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1\n\n5. models\n    (1,1) : category에 해당하는 모델이 없는 경우(예외 처리

- Main Code (주석 해제 후 사용)

In [ ]:
'''
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()
driver.get('https://paperswithcode.com/methods')
data_list = []  
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tmp1=4
while True: # 1,2번   
    try:
        click1_selector = f'body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child({tmp1}) > div > h4 > a'
        temp_categories = driver.find_element(By.CSS_SELECTOR,click1_selector) #
    except NoSuchElementException:       
            break
    driver.execute_script("arguments[0].click();", temp_categories)
    tmp1 +=3
    time.sleep(1.5)
    
    # 클릭으로 인한 페이지 업데이트
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    method_selector = 'body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1'
    method_name = soup.select(method_selector)
    if not method_name:
        break
    method = method_name[0].text.strip().replace('\n','')
    print('method: ',method)
    
    tmp2 = 3
    while True: # 3,4번
        try:
            click2_selector = f'body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child({tmp2}) > a'
            temp_model = driver.find_element(By.CSS_SELECTOR,click2_selector)
        except NoSuchElementException: 
            break
        driver.execute_script("arguments[0].click();", temp_model)
        tmp2 += 3
        time.sleep(1.5)
        
        # 페이지 업데이트
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #here
        category_selector = 'body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1'
        category_name = soup.select(category_selector)
        if not category_name:
            break
        category = category_name[0].text.split("\xa0")[0].strip()
        print('category: ',category)

        model_list = []
        tmp3 = 1
        while True: # 5번
            model_selector = f'#methodsTable > tbody > tr:nth-child({tmp3}) > td:nth-child(1) > div.method-image > a'
            model_name = soup.select(model_selector) 
            if not model_name:
                break
            model = model_name[0].text.strip().replace('\n', '')
            model_list.append(model)
            print("model: ",model)
            tmp3 += 1
            
        data_list.append([method,category,', '.join(model_list)])
        
        driver.back()
    driver.back()

# 데이터를 CSV 파일로 저장
with open('cw_paperswithcode.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['method', 'category', 'model'])  # 헤더 추가
    csv_writer.writerows(data_list)  # 데이터 추가
'''

- csv 구조 변경 (행열 변환)

In [ ]:
import csv

# 입력 파일과 출력 파일 이름\
input_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\cw_paperswithcode.csv'
output_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\paperswithcode.csv'
# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_filename, 'r', newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        data.append(row)

# 데이터를 풀어서 저장할 새로운 리스트 생성
unrolled_data = []
for row in data:
    method = row[0]
    category = row[1]
    models = row[2].split(', ')  # ,를 기준으로 model들을 분리
    for model in models:
        unrolled_data.append([method, category, model])

# 새로운 CSV 파일로 데이터를 쓰기
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in unrolled_data:
        csv_writer.writerow(row)

### 6. 모델 리스트 불러오기 (paperswithcode.csv)

In [38]:
docs2_path = 'vscode/paperswithcode.csv'
docs2 = pd.read_csv(docs2_path)

In [39]:
docs2.info() #model 2개 null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   method    1928 non-null   object
 1   category  1928 non-null   object
 2   model     1926 non-null   object
dtypes: object(3)
memory usage: 45.3+ KB


In [40]:
docs2.describe()

,method,category,model
count,1928,1928,1926
unique,6,226,1350
top,Computer Vision,Image Models,TNT
freq,1203,173,6


In [41]:
docs2

,method,category,model
0,Computer Vision,Image Models,ResNet
1,Computer Vision,Image Models,Vision Transformer
2,Computer Vision,Image Models,VGG
3,Computer Vision,Image Models,DenseNet
4,Computer Vision,Image Models,VGG-16
...,...,...,...
1923,Graphs,Graph Representation Learning,L-GCN
1924,Graphs,Graph Representation Learning,GFSA
1925,Graphs,Graph Representation Learning,DyGED
1926,Graphs,Graph Representation Learning,DeepDrug


### 7. 모델 리스트를 패턴화하여 arxiv dataset에 적용

In [5]:
tmp = docs2[docs2.method.str.contains('Computer Vision')]
model_list = tmp.iloc[:,2].tolist() #모델

model_counts = defaultdict(int) # key : value
patterns={}

# 패턴화 하기
for model in model_list:
    if isinstance(model,str): # model 의 객체가 string 이면
        patterns[model] = re.compile(re.escape(model),re.I)

# document filtering
filtered_cs_cv_docs = {model: cs_cv_docs[cs_cv_docs['abstract'].str.contains(pattern)] 
                      for model,pattern in patterns.items()}

for model,filtered_docs in filtered_cs_cv_docs.items():
    model_counts[model] = len(filtered_docs)

for model,count in model_counts.items():
    print(f"{model} : {count}")
    


NameError: name 'docs2' is not defined

In [47]:
# 결론
# 생각했던 것에 비해 model_counts의 값이 작다.
# 정규표현식을 통한 대소문자 구분 및 문자열 구분 등을 확장하여 개선해본다.

### 8. 정규표현식을 이용하여 7번의 결과를 개선하기

In [48]:
tmp="VGG"

# CASE1 -> GAN : 대소문자 구분 x & 문자열 구분 x -> legan, lovegan (0)
matching_rows1 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(f"{tmp}", re.I))| #정규표현식 re.I -> 대소문자 구별X
    cs_cv_docs['title'].str.contains(re.compile(f"{tmp}", re.I))
]
matching_count1 = len(matching_rows1)
print(matching_count1)


# CASE2 -> 대소문자 구분 o & 문자열 구분 o -> GAN(o) eGAN(x)

matching_rows2 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(rf'\b{tmp}\b')|
    cs_cv_docs['title'].str.contains(rf'\b{tmp}\b')
]
matching_count2 = len(matching_rows2)
print(matching_count2)


# CASE3 -> 대소문자 구분 x & 문자열 구분 o -> GAN & gan (o) -> 영진님 
   
matching_rows3 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b{tmp}\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b{tmp}\b',re.I))
]
matching_count3 = len(matching_rows3)
print(matching_count3)


#CASE4 -> 대소문자 구분 o & 문자열 구분x -> eGAN (o) egan(x)

matching_rows4 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(f"{tmp}")|
    cs_cv_docs['title'].str.contains(f"{tmp}")
]
matching_count4 = len(matching_rows4)
print(matching_count4)

1075
618
640
1040


In [ ]:
'''
<정리 CASE1-4>
1. 대소문자를 구분하지 않아야 함
    UNET,uNet,UNet

2. 문자열 구분은 애매함

2-1) 문자열 구분을 하면(2,3) 모델의 폭이 줄어들음 -> GAN) DCGAN, StyleGAN 불가
하지만 paperswithcode의 모델리스트가 어느정도 커버가 되긴 한다. 

2-2) 문자열 구분을 하지 않으면(1,4) 모델의 폭이 증가하나 CCT모델의 경우 CCTV 단어도 인식을 하기에 리스크가 크다.

'''

In [49]:
# CASE5 -> 대소문자 구분 x & 문자열 구분 * -> 공백 및 숫자 인정 -> ResNet, ResNet 1 , ResNet1 

matching_rows5 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b\d*\s*{tmp}\s*\d*\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b\d*\s*{tmp}\s*\d*\b',re.I))
]
matching_count5 = len(matching_rows5)
print(matching_count5)


# CASE6 -> 대소문자 구분 x & 문자열 구분 * -> 공백 및 숫자 인정 + 's' 인정 -> GANs, VGGs 

matching_rows6 = cs_cv_docs[
    cs_cv_docs['abstract'].str.contains(re.compile(rf'\b\d*\s*{tmp}s?\s*\d*\b',re.I))|
    cs_cv_docs['title'].str.contains(re.compile(rf'\b\d*\s*{tmp}s?\s*\d*\b',re.I))
]
matching_count6 = len(matching_rows6)
print(matching_count6)

929
934


In [ ]:
'''
<정리 CASE5-6>
1. 대소문자는 구분하지 않아야 함.
2. 문자열은 일부 구분
    공백 및 숫자는 허용하도록 함. -> VGG19
3. 모델+'s' 허용
    Abstract 분석 결과 GANs, VGGs 로 표기된 경우도 다수 있었음
'''

### 9. model_list (paperswithcode) + title & abstract (arxiv) 

In [2]:
import pandas as pd
docs3_path = 'vscode/modified_docs1.csv'
docs3 = pd.read_csv(docs3_path)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_27616\3420504382.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  docs3 = pd.read_csv(docs3_path)


In [3]:
docs3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438727 entries, 0 to 438726
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Unnamed: 0                      438727 non-null  int64 
 1   id                              438727 non-null  object
 2   authors                         438727 non-null  object
 3   title                           438727 non-null  object
 4   categories                      438727 non-null  object
 5   created                         438727 non-null  object
 6   last_updated                    438727 non-null  object
 7   submitter                       438599 non-null  object
 8   comments                        268353 non-null  object
 9   journal-ref                     56649 non-null   object
 10  doi                             73663 non-null   object
 11  report-no                       7582 non-null    object
 12  license                       

In [4]:
# docs3[docs3['is_Computer Vision']==True]
docs3.head(50)

,Unnamed: 0,id,authors,title,categories,created,last_updated,submitter,comments,journal-ref,doi,report-no,license,abstract,is_Computer Vision,is_Natural Language Processing,is_Reinforcement Learning,is_Audio,is_Sequential,is_Graph
0,46,704.0047,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 13:06:50 GMT","Sun, 1 Apr 2007 13:06:50 GMT",Igor Grabec,"5 pages, 5 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,The intelligent acoustic emission locator is...,False,False,False,False,False,False
1,49,704.005,T. Kosel and I. Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,"Sun, 1 Apr 2007 18:53:13 GMT","Sun, 1 Apr 2007 18:53:13 GMT",Igor Grabec,"5 pages, 7 eps figures, uses IEEEtran.cls",NaN,NaN,NaN,NaN,Part I describes an intelligent acoustic emi...,False,False,False,False,False,False
2,61,704.0062,"Rastislav \v{S}r\'amek, Bro\v{n}a Brejov\'a, T...",On-line Viterbi Algorithm and Its Relationship...,cs.DS,"Sat, 31 Mar 2007 23:52:33 GMT","Sat, 31 Mar 2007 23:52:33 GMT",Tom\'a\v{s} Vina\v{r},NaN,Algorithms in Bioinformatics: 7th Internationa...,10.1007/978-3-540-74126-8_23,NaN,NaN,"In this paper, we introduce the on-line Vite...",True,True,True,True,True,False
3,89,704.009,Lester Ingber,Real Options for Project Schedules (ROPS),cs.CE cond-mat.stat-mech cs.MS cs.NA physics.d...,"Sun, 1 Apr 2007 14:35:40 GMT","Sun, 1 Apr 2007 14:35:40 GMT",Lester Ingber,NaN,NaN,NaN,Report 2007:ROPS,NaN,Real Options for Project Schedules (ROPS) ha...,False,False,False,False,False,False
4,97,704.0098,"Jack Raymond, David Saad",Sparsely-spread CDMA - a statistical mechanics...,cs.IT math.IT,"Sun, 1 Apr 2007 18:27:26 GMT","Wed, 30 Apr 2008 15:36:55 GMT",Jack Raymond,"23 pages, 5 figures, figure 1 amended since pu...",J. Phys. A: Math. Theor. 40 No 41 (12 October ...,10.1088/1751-8113/40/41/004,NaN,NaN,"Sparse Code Division Multiple Access (CDMA),...",False,False,False,False,False,False
5,107,704.0108,Sergey Gubin,Reducing SAT to 2-SAT,cs.CC,"Sun, 1 Apr 2007 23:16:27 GMT","Sun, 1 Apr 2007 23:16:27 GMT",Sergey Gubin,8 pages,NaN,NaN,NaN,NaN,Description of a polynomial time reduction o...,False,False,False,False,False,False
6,212,704.0213,Ketan D. Mulmuley Hariharan Narayanan,Geometric Complexity Theory V: On deciding non...,cs.CC,"Mon, 2 Apr 2007 15:13:27 GMT","Thu, 27 Sep 2012 15:23:39 GMT",Ketan Mulmuley D,This article has been withdrawn because it has...,NaN,NaN,NaN,NaN,This article has been withdrawn because it h...,False,False,False,False,False,False
7,216,704.0217,Wiroonsak Santipach and Michael L. Honig,Capacity of a Multiple-Antenna Fading Channel ...,cs.IT math.IT,"Mon, 2 Apr 2007 15:35:24 GMT","Tue, 17 Feb 2009 03:49:20 GMT",Wiroonsak Santipach,NaN,"IEEE Trans. Inf. Theory, vol. 55, no. 3, pp. 1...",10.1109/TIT.2008.2011437,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,Given a multiple-input multiple-output (MIMO...,False,False,False,False,False,False
8,217,704.0218,Yuri Pritykin,On Almost Periodicity Criteria for Morphic Seq...,cs.DM cs.LO,"Mon, 2 Apr 2007 15:45:37 GMT","Mon, 2 Apr 2007 15:45:37 GMT",Yuri Pritykin,9 pages. To be presented on 11th International...,NaN,NaN,NaN,NaN,In some particular cases we give criteria fo...,False,False,False,False,False,False
9,228,704.0229,Ketan D. Mulmuley,Geometric Complexity Theory VI: the flip via s...,cs.CC,"Mon, 2 Apr 2007 16:41:38 GMT","Thu, 22 Jan 2009 15:25:24 GMT",Ketan Mulmuley D,139 pages. Corrects error in the conjectural s...,NaN,NaN,NaN,http://arxiv.org/licenses/nonexclusive-distrib...,This article belongs to a series on geometri...,False,False,False,False,False,False


- 중복 칼럼 개수 확인

In [5]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least2 = len(docs3[true_count >= 2])
true_count_least2

212441

In [7]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least3 = len(docs3[true_count >= 3])
true_count_least3

212441

In [8]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least4 = len(docs3[true_count >= 4])
true_count_least4

212441

In [9]:
true_count = docs3[['is_Computer Vision', 'is_Natural Language Processing', 'is_Reinforcement Learning', 'is_Audio','is_Sequential', 'is_Graph']].sum(axis=1)
true_count_least5 = len(docs3[true_count >= 5])
true_count_least5

212441

## test

In [92]:
import pandas as pd
s1 = pd.Series(['house and parrot'])

tmp = 'parrot'
print(s1.str.contains(rf"\b{tmp}\b", case=False))
print(s1.str.match(rf'\b\d\s*{tmp_model}s?\s*\d*\b', case=False))

0    True
dtype: bool
0    False
dtype: bool
